<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/predict_show_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ３着以内の馬を予測

lightGBMでシンプルなモデルを実装する。

- ライブラリ・データ読み込み
- データ前処理
- 学習
- 予測

# ライブラリ・データ読み込み

In [20]:
import numpy as np
import pandas as pd
import re

import lightgbm as lgb

In [13]:
DATA_PATH = '/content/drive/My Drive/data/horse-race/'

In [14]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')

In [15]:
train_df.head()

,age,frame_number,horse_number,jockey,name,popular,race_date,race_id,race_name,weight,win,show,clockwise,distance,field_type,place,race_round,start_time,weather
0,牡3,7.0,13,森泰斗,ワイルドホース,2.0,2020/6/22,202044062212,タイタン賞競走,54.0,3.7,1,右,1400,ダ,大井,12R,20:50,雨
1,牝5,6.0,11,山崎誠士,クインズオライリー,12.0,2020/6/22,202044062212,タイタン賞競走,54.0,81.4,1,右,1400,ダ,大井,12R,20:50,雨
2,セ6,3.0,4,西啓太,メダーリアフレイム,4.0,2020/6/22,202044062212,タイタン賞競走,56.0,8.6,1,右,1400,ダ,大井,12R,20:50,雨
3,牡4,8.0,14,藤本現暉,ゴールドプリンス,8.0,2020/6/22,202044062212,タイタン賞競走,56.0,34.4,0,右,1400,ダ,大井,12R,20:50,雨
4,牝5,4.0,7,藤田凌,イグレット,9.0,2020/6/22,202044062212,タイタン賞競走,54.0,34.8,0,右,1400,ダ,大井,12R,20:50,雨


In [16]:
len(train_df)

253636

# データ前処理

- レースが開催されるのは週末のため日付は利用しない
  - 季節に変換
- 時刻は時間帯に変換
- ageは性別（？）と年齢に分割
- 気温は関係ありそうなため後々追加
- レース名はレースの種類に変換した方が良さそうだが後回し

開催日から季節を追加し、開催日を削除

In [56]:
season = []
for i in range(len(train_df)):
  race_date = train_df.iloc[i]['race_date']
  race_month = int(re.search(r'\/.+?\/', race_date).group().replace('/', ''))
  if 3 <= race_month <= 5:
    season.append('sprint')
  elif 6 <= race_month <= 8:
    season.append('summer')
  elif 9 <= race_month <= 11:
    season.append('autumn')
  else:
    season.append('winter')

In [57]:
train_df['season'] = season

In [59]:
train_df = train_df.drop('race_date', axis=1)

# 学習

# 予測